<a href="https://colab.research.google.com/github/ruchira-net/css-loader/blob/master/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 👇 Installing dependencies { display-mode: "form" }
#@markdown ---
#@markdown Make sure to select **GPU** as the runtime type:<br/>
#@markdown *Runtime->Change Runtime Type->Under Hardware accelerator, select GPU*
#@markdown
#@markdown ---

!pip -q install torch torchvision tor diffusers transformers accelerate scipy safetensors xformers mediapy ipywidgets==7.7.1


In [ ]:
#@title 👇 Selecting Model { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Select Model** - A list of Stable Diffusion models to choose from.
#@markdown - **Select Sampler** - A list of schedulers to choose from. Default is EulerAncestralScheduler.
#@markdown - **Safety Checker** - Enable/Disable uncensored content
#@markdown
#@markdown ---

from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
from diffusers.models import AutoencoderKL
import torch
import ipywidgets as widgets
import importlib
from PIL import Image

#Enable third party widget support
from google.colab import output
output.enable_custom_widget_manager()

#Pipe
pipe = None

#Models
select_model = widgets.Dropdown(
    options=[
        ("Dreamlike Photoreal 2.0" , "stabilityai/stable-diffusion-xl-base-1.0")
    ],
    description="Select Model:"
)

#Schedulers
select_sampler = widgets.Dropdown(
    options=[
        "EulerAncestralDiscreteScheduler",
        "EulerDiscreteScheduler",
        "UniPCMultistepScheduler",
        "DDIMScheduler"
    ],
    description="Select Schedular:"
)
select_sampler.style.description_width = "auto"

#Safety Checker
safety_check = widgets.Checkbox(
    value=False,
    description="Enable Safety Check",
    layout=widgets.Layout(margin="0px 0px 0px -85px")
)

#Output
out = widgets.Output()

#Apply Settings
apply_btn = widgets.Button(
    description="Apply",
    button_style="info"
)


#Get scheduler
def get_scheduler(name):

  match name:

    case "EulerAncestralDiscreteScheduler":
      return EulerAncestralDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")

#Run pipeline
def pipeline(p):

  global pipe

  out.clear_output()
  apply_btn.disabled = True

  with out:

    print("Running, please wait...")

    pipe = StableDiffusionPipeline.from_pretrained(
      select_model.value,
      scheduler=get_scheduler(select_sampler.value),
      torch_dtype=torch.float16,
      vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
    ).to("cuda")

    pipe.safety_checker = None

    pipe.enable_xformers_memory_efficient_attention()

    print("Finished!")

  apply_btn.disabled = False


#Display
apply_btn.on_click(pipeline)

widgets.VBox(
    [
      widgets.HTML(value="<h2>Configure Pipeline</h2>"),
      select_model, select_sampler, safety_check, apply_btn, out
    ]
)

#call pipeline method from here
pipeline(None)

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
#@title 👇 Generating Images { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Prompt** - Description of the image
#@markdown - **Negative Prompt** - Things you don't want to see or ignore in the image
#@markdown - **Steps** - Number of denoising steps. Higher steps may lead to better results but takes longer time to generate the image. Default is `30`.
#@markdown - **CFG** - Guidance scale ranging from `0` to `20`. Lower values allow the AI to be more creative and less strict at following the prompt. Default is `7.5`.
#@markdown - **Seed** - A random value that controls image generation. The same seed and prompt produce the same images. Set `-1` for using random seed values.
#@markdown ---
import ipywidgets as widgets, mediapy, random
import IPython.display


#PARAMETER WIDGETS
width = "300px"

prompt = widgets.Textarea(
    value="(photorealistic (HDR):2), (gorgeous:1.1), (only one:1.4) ",
    placeholder="Enter prompt",
    #description="Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

camera = widgets.Dropdown(
    options=[('leica summicron 35mm f2.0', ' leica summicron 35mm f2.0'), ('sony a7ii', ' sony a7ii'), ('kodak potra', ' kodak potra')],
    value=' leica summicron 35mm f2.0',
    description="Camera:",
    layout=widgets.Layout(width=width)
)

lighting = widgets.Dropdown(
    options=[('romantic lighting', ' romantic lighting,'), ('dramatic lighting', ' dramatic lighting,')],
    value=' dramatic lighting,',
    description="Lighting:",
    layout=widgets.Layout(width=width)
)

neg_prompt = widgets.Textarea(
    value="disfigured, bad lips, ugly lips, bad nose, ugly nose, bad face, ugly face, bad mouth, ugly mouth, beard, bad teeth, ugly teeth, ((grayscale)), cross-eyed, uneven eyes, bad ears, ugly ears, deformed, malformed limbs, bad anatomy, bad hands, three hands, three legs, bad arms, missing legs, missing arms, poorly drawn face, bad face, fused face, cloned face, worst face, three crus, extra crus, fused crus, worst feet, three feet, fused feet, fused thigh, three thigh, fused thigh, extra thigh, worst thigh, missing fingers, extra fingers, ugly fingers, long fingers, horn, extra eyes, huge eyes, 2girl, amputation, disconnected limbs, cartoon, cg, 3d, anime, unreal, animate, masculine, fat, chubby, (tamil:1.7), (red dot:2), (red mark:1.9), (pottu:1.8), (bindi:1.6), paintings, sketches, lowres, ((monochrome)), heavy make-up, (worst quality:1.6), (low quality:1.5), body hair, jewellary, topless, multiple people, blurry, duplicate bodies",
    placeholder="Enter negative prompt",
    #description="Negative Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

num_images = widgets.IntText(
    value=4,
    description="Images:",
    layout=widgets.Layout(width=width),
)

steps = widgets.IntText(
    value=35,
    description="Steps:",
    layout=widgets.Layout(width=width)
)

CFG = widgets.FloatText(
    value=7,
    description="CFG:",
    layout=widgets.Layout(width=width)
)

img_height = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768), ('720px', 720)],
    value=720,
    description="Height:",
    layout=widgets.Layout(width=width)
)

img_width = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768), ('1280px', 1280)],
    value=1280,
    description="Width:",
    layout=widgets.Layout(width=width)
)

random_seed = widgets.IntText(
    value=-1,
    description="Seed:",
    layout=widgets.Layout(width=width),
    disabled=False
)

generate = widgets.Button(
    description="Generate",
    disabled=False,
    button_style="primary"
)

display_imgs = widgets.Output()


#RUN
def generate_img(i):

  #Clear output
  display_imgs.clear_output()
  generate.disabled = True

  #Calculate seed
  seed = random.randint(0, 2147483647) if random_seed.value == -1 else random_seed.value

  with display_imgs:

    print(f"Prompt:\n{prompt.value + lighting.value + camera.value}")

    images = pipe(
      prompt.value + lighting.value + camera.value,
      height = img_height.value,
      width = img_width.value,
      num_inference_steps = steps.value,
      guidance_scale = CFG.value,
      num_images_per_prompt = num_images.value,
      negative_prompt = neg_prompt.value,
      generator = torch.Generator("cuda").manual_seed(seed),
    ).images

    mediapy.show_images(images)

    print(f"Seed:\n{seed}")

  generate.disabled = False

#Display
generate.on_click(generate_img)

widgets.VBox(
    [
      widgets.AppLayout(
        header=widgets.VBox(
            [camera, lighting]
        ),
        left_sidebar=widgets.VBox(
            [num_images, steps, CFG, img_height, img_width, random_seed]
        ),
        center=widgets.VBox(
            [prompt, neg_prompt, generate]
        ),
        right_sidebar=None,
        footer=None
      ),
      display_imgs
    ]
)